import the libraries

In [3]:
# import the libraries we'll need
import pandas as pd
import seaborn as sns
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, recall_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

load the dataset

In [4]:
#file_path = "../../..//data//dataset//change-features.csv"
file_path = "dataset-train.csv"

# Load the CSV file into a pandas DataFrame
data = pd.read_csv(file_path)

# Display the contents of the DataFrame
display(data)

,package,version,PII,file_sys_access,file_process_creation,network_access,cryptographic_functionality,data_encoding,dynamic_code_generation,package_installation,geolocation,minified_code,no_content,longest_line,num_of_files,has_license,label
0,imask-module,0.0.3,0,0,1,0,0,0,0,1,0,0,0,1392,68,1,benign
1,vo-core,1.2.3,0,0,0,1,0,0,0,0,0,0,0,138,10,0,benign
2,flow-bin,0.196.1,0,1,1,0,0,0,0,0,0,1,0,69,14,0,benign
3,notar-cli,1.2.1,1,1,1,1,1,1,1,0,0,0,0,146206,3,0,benign
4,codezist,1.0.0,0,1,1,1,1,1,1,1,0,1,0,592478,3,0,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,testername,1.4.11,0,0,0,0,0,0,0,0,0,0,0,26,2,0,malicious
96,cors.js,2.8.5,0,1,1,1,0,1,0,0,0,0,0,131,8,1,malicious
97,vanshpkg,2.0.0,0,1,1,1,0,1,0,0,0,0,0,109,3,0,malicious
98,touching-fish,0.0.1,0,1,1,1,1,1,0,0,0,0,0,85,17,1,malicious


fit the dataset to the classifiers (Gaussian Naive Bayes, Decision Tree and SVM). Then perform voting

In [5]:
X_train = data.drop(["package", "version", "label"], axis=1)
y_train = data["label"]

# Train individual classifiers
dtc = DecisionTreeClassifier(random_state=42)
dtc.fit(X_train, y_train)

nb = GaussianNB()
nb.fit(X_train, y_train)

svm = SVC(random_state=42)
svm.fit(X_train, y_train)

# Combine the classifiers using a voting classifier
voting_clf = VotingClassifier(
    estimators=[('dtc', dtc), ('nb', nb), ('svm', svm)],
    voting='hard'
)

voting_clf.fit(X_train, y_train)




VotingClassifier(estimators=[('dtc', DecisionTreeClassifier(random_state=42)),
                             ('nb', GaussianNB()),
                             ('svm', SVC(random_state=42))])

Validate the voting classifier

In [6]:
val_data = pd.read_csv("dataset-validationSrc.csv")
X_val = val_data.drop(["package", "version", "label"], axis=1)
y_val = val_data["label"]

In [7]:
y_pred_val = voting_clf.predict(X_val)
score_val = voting_clf.score(X_val, y_val)
recall_val = recall_score(y_val, y_pred_val, pos_label='malicious')
print("Validation set accuracy: {:.2f}%".format(score_val * 100))
print("Validation set recall: {:.2f}%".format(recall_val * 100))

# Print the confusion matrix for the validation set
cm = confusion_matrix(y_val, y_pred_val)
print("Validation set confusion matrix:")
print(cm)

dtc.fit(X_val, y_val)

Validation set accuracy: 67.39%
Validation set recall: 100.00%
Validation set confusion matrix:
[[ 8 15]
 [ 0 23]]


DecisionTreeClassifier(random_state=42)

Export the training data in a pkl file

In [8]:
import pickle
pickle.dump(dtc, open('model.pkl','wb'))

Test

In [9]:
import pandas as pd
import pickle

# Load the saved model from the .pkl file
with open('model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

# Define a function to classify a package
def classify_package(package_features):
    # Create a DataFrame with the package features
    df = pd.DataFrame([package_features])
    # df.head(5)
    # Assuming 'label' is the target column you want to predict
    # Extract the features (excluding the 'label' column)
    # X = df.drop(columns=['label'])

    # Make predictions
    prediction = model.predict(df)
    return prediction
    # Convert the prediction to a human-readable label (e.g., 'benign' or 'malicious')
    # if predictions[0] == 0:
    #     return 'benign'
    # else:
    #
#0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 103, 11, 1
package_features3 = {
    'PII': 0,
    'file_sys_access': 0,
    'file_process_creation': 0,
    'network_access': 0,
    'cryptographic_functionality': 0,
    'data_encoding': 0,
    'dynamic_code_generation': 0,
    'package_installation': 0,
    'geolocation': 0,
    'minified_code': 0,
    'no_content': 0,
    'longest_line': 103,
    'num_of_files': 11,
    'has_license': 1
}
print(classify_package(package_features3))


['benign']
